In [ ]:
import pandas as pd
import numpy as np
#import tiingo
import requests


#### Remove holdiays from timeseries

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()

holidays = cal.holidays(start='2000-01-01',
                        end='2030-12-30')

holiday_df = pd.DataFrame(data=holidays,
                          columns = ['date']).assign(holiday='USFederalHoliday')

holiday_df.rename(columns={'date':'ds'},inplace=True)
#holiday_df.to_csv('us_holidays_until_2030.csv')

In [ ]:
### First we will get the 15min time intervals and calculate the 15min Moving averages (10 perido, 20 period, 50 period)
### From the same data we will also calculate the 1d Moving averages (10 period, 20 period, 50 period) and
### Append the indicators to the dataframe 
### Each iteration will run for a single ticker


### The data extracts work in 0GMT => Exchange opens at 14:30 and closes at 21:00
### Convert this to Sofia/New-York time if needed


#### Connect to tiingo

In [ ]:
# 15min to # 1h 
headers = {
    'Content-Type': 'application/json'
}
requestResponse = requests.get("https://api.tiingo.com/iex/SPY/prices?startDate=2015-01-02&endDate=2022-12-31&resampleFreq=5min&columns=open,high,low,close,volume&token=0ed744c7db5ef348139953912e2f3cee79f4608b", headers=headers)
#print(requestResponse.json())

In [ ]:
# # 15min to # 1h 
# headers = {
#     'Content-Type': 'application/json'
# }
# requestResponse = requests.get("https://api.tiingo.com/tiingo/daily/BB/prices?startDate=2016-01-01&token=0ed744c7db5ef348139953912e2f3cee79f4608b", headers=headers)
# #print(requestResponse.json())

In [ ]:
# ## The earliest Tiingo 5 min data starts from 2016-12-01 
# ticker = 'SPY'

# # Generate month_start and month end to use for looping over and getting data month by month
# month_start_date  = pd.date_range(start='12/1/2016', end='12/1/2022', freq='MS').strftime('%Y-%m-%d')
# month_end_date  = pd.date_range(start='12/1/2016', end='12/31/2022', freq='M').strftime('%Y-%m-%d')
# print("the size of the monthly data is " + str(month_end_date.size))


# total_df = pd.DataFrame()
# for x in range(month_start_date.size):
#     month_start = month_start_date[x]
#     month_end  = month_end_date[x]
#     requestResponse = requests.get("https://api.tiingo.com/iex/"+str(ticker)+"/prices?startDate="+str(month_start)+"&endDate="+str(month_end)+"&resampleFreq=5min&columns=open,high,low,close,volume&token=0ed744c7db5ef348139953912e2f3cee79f4608b", headers=headers)
#     print(x) 
#     #print("https://api.tiingo.com/iex/SPY/prices?startDate="+str(month_start)+"&endDate="+str(month_end)+"&resampleFreq=5min&columns=open,high,low,close,volume&token=0ed744c7db5ef348139953912e2f3cee79f4608b") 
#     df_stock = pd.DataFrame(requestResponse.json()) 

#     total_df = total_df.append(df_stock)

# print('Data downloading ddone for the ticker '+str(ticker))
# total_df.to_csv('spy_5_min_all_data.csv')

In [ ]:
# 15min to # 1h 
headers = {
    'Content-Type': 'application/json'
}
requestResponse = requests.get("https://api.tiingo.com/iex/SPY/prices?startDate=2021-01-02&endDate=2021-02-01&resampleFreq=5min&columns=open,high,low,close,volume&token=0ed744c7db5ef348139953912e2f3cee79f4608b", headers=headers)
#print(requestResponse.json())

In [ ]:
df_stock = pd.DataFrame(requestResponse.json()) 
df_stock

In [ ]:
## This gets financial data
#requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/msft/statements?token=0ed744c7db5ef348139953912e2f3cee79f4608b", headers=headers)
#print(requestResponse.json)

In [ ]:
requestResponse

In [ ]:
df_stock = pd.DataFrame(requestResponse.json()) 

In [ ]:
# ## Implement shitloads of historical data if want
# total_df = pd.read_csv('spy_5_min_all_data.csv')
# df_stock = total_df

In [ ]:
df_stock

In [ ]:
df_stock.date = pd.to_datetime(df_stock.date)

In [ ]:
df_stock.date.dt.date

In [ ]:
## Remove the US public holidays
df_stock = df_stock[~df_stock.date.dt.date.isin(holiday_df.ds)]

In [ ]:
df_stock.dtypes

#### Prophet

In [ ]:
from prophet import Prophet

In [ ]:
df_stock.rename(columns={"date": "ds", "close": "y"},inplace=True)

In [ ]:
df_stock

In [ ]:
df_stock.ds = df_stock.ds.dt.tz_convert(None)

In [ ]:
df_stock.ds.dt.day_name().value_counts()

In [ ]:
# Python
m = Prophet(holidays=holiday_df, growth = 'linear')
# m = Prophet(holidays=holiday_df, growth = 'linear')
m.fit(df_stock[['ds','y']])

In [ ]:
# Python
future = m.make_future_dataframe(periods=100, freq='5min')
future.tail()

#### Prediction results

In [ ]:
# Python
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(100)

In [ ]:
# Python
fig1 = m.plot(forecast)

In [ ]:
# Python
fig2 = m.plot_components(forecast)

In [ ]:
# Python
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [ ]:
# Python
plot_components_plotly(m, forecast)

In [ ]:
top_by_volume = pd.read_csv('top_by_volume.csv')

In [ ]:
top_by_volume.Ticker.values

In [ ]:
# import xgboost as xgb 
# clf_xgb = xgb.XGBRegressor(objective='reg:squarederror', 
#                             tree_method = 'gpu_hist',
#                             gamma = 0.5,
#                            max_depth = 25,
#                             seed=42,
#                             n_estimators=30)

In [ ]:
# clf_xgb.fit(X_train,
#             y_train,
#            verbose=True,
#             early_stopping_rounds=20,
#             eval_metric='rmse',   #aucpr,auc, map
#             eval_set=[(X_test,y_test)])